<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/06_nearest_neighbors_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k近傍法（続）

## 6.1 準備

### 6.1.1 インポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

### 6.1.2 データファイルのアップロード

* `lifesat_extended.csv`をアップロードする。

In [ ]:
from google.colab import files
files.upload()

### 6.1.3 データファイルの読み込み
* 今回は、ファイルを読み込むときに、国名をインデックスに設定してしまう。
 * index_col引数を使っている。

In [ ]:
df = pd.read_csv('lifesat_extended.csv', index_col=0)

In [ ]:
df

* 日本をtest setとして除外し、残りのデータ集合を使う。
 * 日本の生活満足度を予測することを、最終的な目標としたのだった。

In [ ]:
df_test = df.loc[['Japan']]
df = df.drop(['Japan'])

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
df.plot(kind='scatter', x='GDP per capita', y='Life satisfaction', ax=ax[0])
df.plot(kind='scatter', x='Employment rate', y='Life satisfaction', ax=ax[1]);

In [ ]:
X = df[['GDP per capita', 'Employment rate']]
y = df['Life satisfaction']

In [ ]:
X

## 6.2 二つの特徴量を同時に使う
* 前回は、一人当たりのGDPと、雇用率を、別々に使った。
* 今回は、これら二つの特徴量を、同時に使いたい。
* つまり、(一人当たりのGDP, 雇用率)という2次元ベクトルを使って、生活満足度を予測したい。

### 6.2.1 演習

* 演習問題6-1: 韓国とイタリアの距離を、一人当たりのGDPと雇用率を同時に使って計算したい。しかし、下に示す距離の計算方法には、問題がある。どのような問題があるか。

In [ ]:
print(np.linalg.norm(X.loc['Korea'] - X.loc['Italy']))

* 演習問題6-2: 上で見つけた問題を解決した上で、あらためて、韓国とイタリアの距離を求めてみよう。

## 6.3 訓練データ/検証データ/テストデータ


### 6.3.1 テストデータ
* 最終的にそれについて予測を行なうことで、手法の評価をおこないたいデータを、テストデータと呼ぶ。
* よって、今回は、日本のデータが、テストデータになる。
 * 普通は、テストデータは今回のように一つだけではなく、複数、用意する。
 * 今回は、データ数そのものが29個と少ないため、テストデータは一つだけにしている。

### 6.3.2 検証データ

* 最適な近傍の個数kを、どうやって求めたらいいだろうか。
* kの値を変えながら、日本の生活満足度を予測し、予測誤差を見ながらkを決めるのでは、カンニングしていることになる。
 * テストデータでの評価は、最終評価なので、最後に一度行うだけ。
 * つまり、テストデータについては、答えが分かっていないと想定する。
* 日本の生活満足度を予測する前に、どのようにしてkを決めればいいだろうか。


* そこで、テストデータ以外のデータを使って、予測問題を解く。

* テストデータ以外のデータの中から、国を一つ選んで（例えば韓国）、その国について予測をしてみる。
* k近傍法の場合のkのように、チューニングしなければならないパラメータを決めるために使うデータを、検証データと呼ぶ。
 * 本当にそれについて予測をしたいデータは、テストデータ。
 * 検証(validation)のためにそれについて予測をおこなうデータが、検証データ。
* 検証データについて予測を行うことで、kの値を変えたとき予測性能がどう変わるか、調べる。
 * 検証データも、普通は、複数個用意する。
 * 今回は、データ数そのものが少ないため、検証データも一つだけにしている。


### 6.3.3 訓練データ
* 特定のインスタンスについて予測を実行するために使うデータを、訓練データと呼ぶ。
* k近傍法では、そのなかから、特定の国の近傍をk個見つけてくるデータが、訓練データになる。

## 6.4 検証データを使った最適なkの決定
* ここでは、韓国を検証データとして使う。
 * 日本はテストデータなので、最初に除外しておいた。
* そして、韓国の生活満足度を予測し、最も良い予測値を与えるkがいくらか、調べる。

### 6.4.1 演習

* 演習問題6-3: 韓国の生活満足度を予測し、最も良い予測値を与えるkを調べよう。

## 6.5 ‎leave-one-out交差検証 (1)
* 上では、韓国を検証データとして使った。
* しかし、韓国以外の国を検証データとしても構わないはずである。
* そこで、テストデータである日本以外の国について、その一つ一つを検証データとした場合の、それぞれの最適なkの値を求めてみる。

### 6.5.1 演習

* 演習問題6-4: 上で韓国について行ったことと同じことを、他の国についても実行し、最適なkの値がどのくらい違ってくるものか、見てみよう。

* 最適なkを、どのようにして決めればいいだろうか？
 * 例えば、各国について得られたkの平均をとることが考えられるが、これは良い方法と言えるだろうか？

## 6.6 ‎leave-one-out交差検証 (2)
* 上では、テストデータである日本を除く国を、一つずつ順番に検証データとして、最適なkを求めた。
* しかし、こうして国ごとに求められたkを、どうやってまとめあげるのか、疑問が残った。
 * それらkの平均をとる、という方法は、何かおかしい感じがする。
 * 最頻値の方が、まだましであろう。
* 最適なkを決める、もっと良い方法はないだろうか？

### 6.6.1 演習

* 演習問題6-5: 韓国を検証データとする。つまり、韓国について予測を行なう。k=1から27まで変えたとき、それぞれ予測誤差がいくらになるか、求めてみよう。

* 演習問題6-6: 縦が国名、横がkの値の表を、データフレームとして作り、日本を除く28カ国の国のひとつひとつを検証データとしたとき、各々のkの値で予測誤差がいくらになるかで埋めてみよう。

* 演習問題6-7: 上で作った表で、それぞれのkの値について、予測誤差の平均値を求めてみよう。

* 演習問題6-8: 上で求めた予測誤差の平均値を、最も小さくするkの値は、いくらになるか。

* 演習問題6-9: こうして求めた最適なkを使って、最終的に、日本の生活満足度の予測をしてみよう。